<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Redes recurrentes, implementación simple

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 9 de mayo de 2018.

En esta libreta vamos a explorar como desarrollar redes recurrentes, simples y basadas en unidades de memoria de largo y corto plazo (LSTM), aplicadas a la generación automática de texto.

Dado que estamos en México en año electoral, y que se vota por diputados y presidentes municipales en muchisimos municipios del país, nos preguntamos si podríamos inventar nuevos municipios para que todos los candidatos tuvieran un lugar que gobernar. Así, generamos una lista con el nombre de todos los municipios de México, y la vamos a usar para aprender los nombres, y generar nombres a partir de una red recurrente. Esto es interesante ya que en México hay muchos municipios cuyos nombres tienen raices del español, el nahuatl, direrentes lenguas mayas, varias lenguas de la familia yuto-azteca, e inclusive algunos que son palabras inventadas (como Mexicali). Así que generar nombres de municipios mexicanos *creibles* es un problema interesante.

El archivo con el nombre de los municipios se encuentra para su descarga [aqui](../varios/municipios.txt).


## 1. Redes recurrentes: Desarrollar una red recurrente completamente a pie.

Con el fin de entender como funcionan las redes neuronales, vamos a aplicar un modelo de generaci´pn de texto *letra a letra*, en el cual tanto la arquitectura como el método de aprendizaje sean programados a mano. 

No vamos a pedir que lo programen todo solos, simplemente que utilicen el modelo programados en este [gist](https://gist.github.com/karpathy/d4dee566867f8291f086), y lo adapten a leer el nombre de los municipios y a generar nombres de municipios.

Para esto:

1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Copiar y comentar en español el contenido del método de verificción de gradiente (para limpiar el código de errores) y usarlo por unas cuantas iteraciones para demostrar que el algoritmo de entrenamiento funciona correctamente.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [1]:
import numpy as np

In [2]:
def F_pérdida(inputs, targets, hprev):
    """
    :inputs
        Arreglo de enteros
    :targets
        Arreglo de enteros
    :hprev
        Arreglo de Hx1 del estado oculto inicial.
    Regresa ->
    El valor de pérdida, los gradientes de los parámetros del modelo y el estado de la última capa oculta
     """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [3]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [4]:
# Lectura de datos
data = open('municipios.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 35530 characters, 68 unique.


In [5]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [6]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [7]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [8]:
while n < 10005:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    #print('inputs: ', inputs)
    #print('targets: ', targets)
    
    # sample from the model now and then
    if n % 2500 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = F_pérdida(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 1000 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 dv,nZÁMsígÑÁ,adóOOÁñ20XlroIBY.eLMpbZ0SñiíuRbOCzxóB6mEdqFg,IzétAvDYYjTVyP yYVSmQiT6nx
ólN uv
sÑdCEFnUCXmútycAAqX,SijúDRkSrUGUqyS,bfYUp0bPxTnS
mv.Gip,Ñ2msOxkqYóSpy.dVFqDOPOpMXzGrúóCqsuvAInf8-SbDj L.DsPu 
----
iter 0, loss: 105.487693
iter 1000, loss: 86.325000
iter 2000, loss: 72.822437
----
 ec
San Mamárdal
Santa Tatoelpal Atríc
Santlanteca Santa Sanuahame
Santa Sahuíca
Saní,ida A
San
Santo loVítKartluit dloliríetuitatarissonaohan diago
Sanua
Sagia Laljuateg
San
Eéctaozíadguhda Parar Sane 
----
iter 3000, loss: 63.077324
iter 4000, loss: 57.006088
----
 ntra dela
Sanatas Ari dóno
Sanyjantacmo Mon
Huútec Bahorho de Arér Pinill Hucara
San Terda lin pareg Nos
SentlqhYalta Eahiran
SaSaradolo
SanCula
San FCómo Quapociatupos
San Flonayo
deteto Yechuaco
Sin 
----
iter 5000, loss: 58.067927
iter 6000, loss: 55.456146


KeyboardInterrupt: 

## Redes recurrentes tipo LSTM

Las redes con unidades LSTM las vimos platicadas en clase, pero no hay nada mejor para entender un tema que implementarlo y comparar los resultados con la red recurrente simple, sin memoria, para esto vamos a hacer lo mismo que antes, pero con unidades LSTM.

Para esto vamos a utilizar otro [gist](https://gist.github.com/karpathy/587454dc0146a6ae21fc) del mismo autor (que es una referencia obligada en el tema, por cierto). En este *gist*, el autor presenta un modelo de redes recurrentes LSTM desarrollado con *numpy* incluido el método de entrenamiento, pero no lo aplica a el modelado *letra a letra* como el gist pasado. Para esta parte de la libreta, lo que tienen que realizar es lo siguiente:


1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Adaptar el modelo propuesto para usarlo en la generación de nombres de municipios.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [10]:
"""
This is a batched LSTM forward and backward pass
"""
import numpy as np
import code

In [41]:
class LSTM:
  
  @staticmethod
  def init(input_size, hidden_size, fancy_forget_bias_init = 3):
    """ 
    Initialize parameters of the LSTM (both weights and biases in one matrix) 
    One might way to have a positive fancy_forget_bias_init number (e.g. maybe even up to 5, in some papers)
    """
    # +1 for the biases, which will be the first row of WLSTM
    WLSTM = np.random.randn(input_size + hidden_size + 1, 4 * hidden_size) / np.sqrt(input_size + hidden_size)
    WLSTM[0,:] = 0 # initialize biases to zero
    if fancy_forget_bias_init != 0:
      # forget gates get little bit negative bias initially to encourage them to be turned off
      # remember that due to Xavier initialization above, the raw output activations from gates before
      # nonlinearity are zero mean and on order of standard deviation ~1
      WLSTM[0,hidden_size:2*hidden_size] = fancy_forget_bias_init
    return WLSTM
  
  @staticmethod
  def forward(X, WLSTM, c0 = None, h0 = None):
    """
    X should be of shape (n,b,input_size), where n = length of sequence, b = batch size
    """
    n,b,input_size = X.shape
    d = int(WLSTM.shape[1]/4) # hidden size
    if c0 is None: c0 = np.zeros((b,d))
    if h0 is None: h0 = np.zeros((b,d))
    
    # Perform the LSTM forward pass with X as the input
    xphpb = WLSTM.shape[0] # x plus h plus bias, lol
    Hin = np.zeros((n, b, xphpb)) # input [1, xt, ht-1] to each tick of the LSTM
    Hout = np.zeros((n, b, d)) # hidden representation of the LSTM (gated cell content)
    IFOG = np.zeros((n, b, d * 4)) # input, forget, output, gate (IFOG)
    IFOGf = np.zeros((n, b, d * 4)) # after nonlinearity
    C = np.zeros((n, b, d)) # cell content
    Ct = np.zeros((n, b, d)) # tanh of cell content
    for t in range(n):
      # concat [x,h] as input to the LSTM
      prevh = Hout[t-1] if t > 0 else h0
      Hin[t,:,0] = 1 # bias
      Hin[t,:,1:input_size+1] = X[t]
      Hin[t,:,input_size+1:] = prevh
      # compute all gate activations. dots: (most work is this line)
      IFOG[t] = Hin[t].dot(WLSTM)
      # non-linearities
      IFOGf[t,:,:int(3*d)] = 1.0/(1.0+np.exp(-IFOG[t,:,:int(3*d)])) # sigmoids; these are the gates
      IFOGf[t,:,3*d:] = np.tanh(IFOG[t,:,3*d:]) # tanh
      # compute the cell activation
      prevc = C[t-1] if t > 0 else c0
      C[t] = IFOGf[t,:,:d] * IFOGf[t,:,3*d:] + IFOGf[t,:,d:2*d] * prevc
      Ct[t] = np.tanh(C[t])
      Hout[t] = IFOGf[t,:,2*d:3*d] * Ct[t]

    cache = {}
    cache['WLSTM'] = WLSTM
    cache['Hout'] = Hout
    cache['IFOGf'] = IFOGf
    cache['IFOG'] = IFOG
    cache['C'] = C
    cache['Ct'] = Ct
    cache['Hin'] = Hin
    cache['c0'] = c0
    cache['h0'] = h0

    # return C[t], as well so we can continue LSTM with prev state init if needed
    return Hout, C[t], Hout[t], cache
  
  @staticmethod
  def backward(dHout_in, cache, dcn = None, dhn = None): 

    WLSTM = cache['WLSTM']
    Hout = cache['Hout']
    IFOGf = cache['IFOGf']
    IFOG = cache['IFOG']
    C = cache['C']
    Ct = cache['Ct']
    Hin = cache['Hin']
    c0 = cache['c0']
    h0 = cache['h0']
    n,b,d = Hout.shape
    input_size = WLSTM.shape[0] - d - 1 # -1 due to bias
 
    # backprop the LSTM
    dIFOG = np.zeros(IFOG.shape)
    dIFOGf = np.zeros(IFOGf.shape)
    dWLSTM = np.zeros(WLSTM.shape)
    dHin = np.zeros(Hin.shape)
    dC = np.zeros(C.shape)
    dX = np.zeros((n,b,input_size))
    dh0 = np.zeros((b, d))
    dc0 = np.zeros((b, d))
    dHout = dHout_in.copy() # make a copy so we don't have any funny side effects
    if dcn is not None: dC[n-1] += dcn.copy() # carry over gradients from later
    if dhn is not None: dHout[n-1] += dhn.copy()
    for t in reversed(range(n)):
 
      tanhCt = Ct[t]
      dIFOGf[t,:,2*d:3*d] = tanhCt * dHout[t]
      # backprop tanh non-linearity first then continue backprop
      dC[t] += (1-tanhCt**2) * (IFOGf[t,:,2*d:3*d] * dHout[t])
 
      if t > 0:
        dIFOGf[t,:,d:2*d] = C[t-1] * dC[t]
        dC[t-1] += IFOGf[t,:,d:2*d] * dC[t]
      else:
        dIFOGf[t,:,d:2*d] = c0 * dC[t]
        dc0 = IFOGf[t,:,d:2*d] * dC[t]
      dIFOGf[t,:,:d] = IFOGf[t,:,3*d:] * dC[t]
      dIFOGf[t,:,3*d:] = IFOGf[t,:,:d] * dC[t]
      
      # backprop activation functions
      dIFOG[t,:,3*d:] = (1 - IFOGf[t,:,3*d:] ** 2) * dIFOGf[t,:,3*d:]
      y = IFOGf[t,:,:3*d]
      dIFOG[t,:,:3*d] = (y*(1.0-y)) * dIFOGf[t,:,:3*d]
 
      # backprop matrix multiply
      dWLSTM += np.dot(Hin[t].transpose(), dIFOG[t])
      dHin[t] = dIFOG[t].dot(WLSTM.transpose())
 
      # backprop the identity transforms into Hin
      dX[t] = dHin[t,:,1:input_size+1]
      if t > 0:
        dHout[t-1,:] += dHin[t,:,input_size+1:]
      else:
        dh0 += dHin[t,:,input_size+1:]
 
    return dX, dWLSTM, dc0, dh0



In [42]:
# -------------------
# TEST CASES
# -------------------



def checkSequentialMatchesBatch():
  """ check LSTM I/O forward/backward interactions """

  n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
  input_size = 10
  WLSTM = LSTM.init(input_size, d) # input size, hidden size
  X = np.random.randn(n,b,input_size)
  h0 = np.random.randn(b,d)
  c0 = np.random.randn(b,d)

  # sequential forward
  cprev = c0
  hprev = h0
  caches = [{} for t in range(n)]
  Hcat = np.zeros((n,b,d))
  for t in range(n):
    xt = X[t:t+1]
    _, cprev, hprev, cache = LSTM.forward(xt, WLSTM, cprev, hprev)
    caches[t] = cache
    Hcat[t] = hprev

  # sanity check: perform batch forward to check that we get the same thing
  H, _, _, batch_cache = LSTM.forward(X, WLSTM, c0, h0)
  assert np.allclose(H, Hcat), 'Sequential and Batch forward don''t match!'

  # eval loss
  wrand = np.random.randn(*Hcat.shape)
  loss = np.sum(Hcat * wrand)
  dH = wrand

  # get the batched version gradients
  BdX, BdWLSTM, Bdc0, Bdh0 = LSTM.backward(dH, batch_cache)

  # now perform sequential backward
  dX = np.zeros_like(X)
  dWLSTM = np.zeros_like(WLSTM)
  dc0 = np.zeros_like(c0)
  dh0 = np.zeros_like(h0)
  dcnext = None
  dhnext = None
  for t in reversed(range(n)):
    dht = dH[t].reshape(1, b, d)
    dx, dWLSTMt, dcprev, dhprev = LSTM.backward(dht, caches[t], dcnext, dhnext)
    dhnext = dhprev
    dcnext = dcprev

    dWLSTM += dWLSTMt # accumulate LSTM gradient
    dX[t] = dx[0]
    if t == 0:
      dc0 = dcprev
      dh0 = dhprev

  # and make sure the gradients match
  print ('Making sure batched version agrees with sequential version: (should all be True)')
  print (np.allclose(BdX, dX))
  print (np.allclose(BdWLSTM, dWLSTM))
  print (np.allclose(Bdc0, dc0))
  print (np.allclose(Bdh0, dh0))
  

In [43]:
def checkBatchGradient():
  """ check that the batch gradient is correct """

  # lets gradient check this beast
  n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
  input_size = 10
  WLSTM = LSTM.init(input_size, d) # input size, hidden size
  X = np.random.randn(n,b,input_size)
  h0 = np.random.randn(b,d)
  c0 = np.random.randn(b,d)

  # batch forward backward
  H, Ct, Ht, cache = LSTM.forward(X, WLSTM, c0, h0)
  wrand = np.random.randn(*H.shape)
  loss = np.sum(H * wrand) # weighted sum is a nice hash to use I think
  dH = wrand
  dX, dWLSTM, dc0, dh0 = LSTM.backward(dH, cache)

  def fwd():
    h,_,_,_ = LSTM.forward(X, WLSTM, c0, h0)
    return np.sum(h * wrand)

  # now gradient check all
  delta = 1e-5
  rel_error_thr_warning = 1e-2
  rel_error_thr_error = 1
  tocheck = [X, WLSTM, c0, h0]
  grads_analytic = [dX, dWLSTM, dc0, dh0]
  names = ['X', 'WLSTM', 'c0', 'h0']
  for j in range(len(tocheck)):
    mat = tocheck[j]
    dmat = grads_analytic[j]
    name = names[j]
    # gradcheck
    for i in range(mat.size):
      old_val = mat.flat[i]
      mat.flat[i] = old_val + delta
      loss0 = fwd()
      mat.flat[i] = old_val - delta
      loss1 = fwd()
      mat.flat[i] = old_val

      grad_analytic = dmat.flat[i]
      grad_numerical = (loss0 - loss1) / (2 * delta)

      if grad_numerical == 0 and grad_analytic == 0:
        rel_error = 0 # both are zero, OK.
        status = 'OK'
      elif abs(grad_numerical) < 1e-7 and abs(grad_analytic) < 1e-7:
        rel_error = 0 # not enough precision to check this
        status = 'VAL SMALL WARNING'
      else:
        rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
        status = 'OK'
        if rel_error > rel_error_thr_warning: status = 'WARNING'
        if rel_error > rel_error_thr_error: status = '!!!!! NOTOK'

      # print stats
      print ('%s checking param %s index %s (val = %+8f), analytic = %+8f, numerical = %+8f, relative error = %+8f' 
            % (status, name, np.unravel_index(i, mat.shape), old_val, grad_analytic, grad_numerical, rel_error))


In [46]:
  checkSequentialMatchesBatch()
  # raw_input('check OK, press key to continue to gradient check')
  print ('every line should start with OK. Have a nice day!')
  checkBatchGradient()
  print ('every line should start with OK. Have a nice day!')

Making sure batched version agrees with sequential version: (should all be True)
True
True
True
True
every line should start with OK. Have a nice day!
OK checking param X index (0, 0, 0) (val = -2.491962), analytic = -0.159347, numerical = -0.159347, relative error = +0.000000
OK checking param X index (0, 0, 1) (val = -0.051318), analytic = +0.322181, numerical = +0.322181, relative error = +0.000000
OK checking param X index (0, 0, 2) (val = -0.731452), analytic = +0.269389, numerical = +0.269389, relative error = +0.000000
OK checking param X index (0, 0, 3) (val = +0.110885), analytic = -0.121545, numerical = -0.121545, relative error = +0.000000
OK checking param X index (0, 0, 4) (val = -0.666297), analytic = +0.060228, numerical = +0.060228, relative error = +0.000000
OK checking param X index (0, 0, 5) (val = +0.383509), analytic = +0.135720, numerical = +0.135720, relative error = +0.000000
OK checking param X index (0, 0, 6) (val = -0.399913), analytic = +0.204151, numerical 

OK checking param X index (4, 0, 8) (val = -1.241441), analytic = +0.041727, numerical = +0.041727, relative error = +0.000000
OK checking param X index (4, 0, 9) (val = -0.425563), analytic = -0.068699, numerical = -0.068699, relative error = +0.000000
OK checking param X index (4, 1, 0) (val = -0.448645), analytic = -0.028436, numerical = -0.028436, relative error = +0.000000
OK checking param X index (4, 1, 1) (val = -0.454397), analytic = +0.077134, numerical = +0.077134, relative error = +0.000000
OK checking param X index (4, 1, 2) (val = -0.523502), analytic = +0.002873, numerical = +0.002873, relative error = +0.000000
OK checking param X index (4, 1, 3) (val = -0.811077), analytic = -0.032419, numerical = -0.032419, relative error = +0.000000
OK checking param X index (4, 1, 4) (val = +2.287059), analytic = +0.025574, numerical = +0.025574, relative error = +0.000000
OK checking param X index (4, 1, 5) (val = -1.758771), analytic = +0.156483, numerical = +0.156483, relative er

OK checking param WLSTM index (9, 4) (val = +0.018605), analytic = +0.590581, numerical = +0.590581, relative error = +0.000000
OK checking param WLSTM index (9, 5) (val = +0.535201), analytic = -0.010496, numerical = -0.010496, relative error = +0.000000
OK checking param WLSTM index (9, 6) (val = +0.268557), analytic = +0.025794, numerical = +0.025794, relative error = +0.000000
OK checking param WLSTM index (9, 7) (val = +0.450454), analytic = -0.147298, numerical = -0.147298, relative error = +0.000000
OK checking param WLSTM index (9, 8) (val = +0.244115), analytic = -0.451157, numerical = -0.451157, relative error = +0.000000
OK checking param WLSTM index (9, 9) (val = -0.092298), analytic = +0.091207, numerical = +0.091207, relative error = +0.000000
OK checking param WLSTM index (9, 10) (val = +0.113931), analytic = -0.320714, numerical = -0.320714, relative error = +0.000000
OK checking param WLSTM index (9, 11) (val = +0.056826), analytic = -0.724422, numerical = -0.724422, r

Por último, agrega en esta misma celda comentarios sobre la comparación entre los dos modelos vistos, diferencias, similitudes, ventajas, desventajas. Recuerda es una opinión personal basada en tu trabajo, no pongas lo que dice la literatura si no lo que tu experimentaste a la hora de desarrollar y aplicar los modelos, así no concuerde con lo que leas en otros lados.